In [2]:
import geopandas as gpd
import pandas as pd

In [8]:

import re

from geopy.geocoders import ArcGIS
#from geopy.extra.rate_limiter import RateLimiter

import plotly.express as px 


#import nltk

import spacy
from spacy import displacy



ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
text = "corrido corpus\ElCorridodeGregorioCortez_X.txt"
with open(text, 'r', encoding='utf-8') as c:
    text = c.read()
    
def clean_text(text):
    cleaned= re.sub(r'[":;,.“”]', "", text)
    return(cleaned)
text = clean_text(text)
#print(text)


TxGPE=[]
nlp = spacy.load("tx_ner_model")
doc =nlp(text) 
#print(doc)
for ent in doc.ents:
    #print(ent.text, ent.label_)
    if ent.label_ == "GPE":
        TxGPE.append(ent.text)
print(TxGPE)

In [ ]:
ents = [(e.text, e.start_char, e.end_char, e.label_)for e in doc.ents]
print(ents)

ents = [(e.text)for e in doc.ents]
print(ents)

In [ ]:
displacy.render(doc, style ='ent', jupyter=True, page=True)

Geoparsing: Finding places from the corrido lyrics

In [ ]:
df = pd.DataFrame(ents, columns=["NER_Places"])
geolocator = ArcGIS(user_agent='CorridosMap')
geocode = lambda query: geolocator.geocode("%s, Texas" % query)
df['Location'] = df['NER_Places'].apply(geocode)
#df['geometry'] = df['NER_Places'].apply(geocode)
df

In [ ]:
gdf = gpd.tools.geocode(df.Location, provider='ArcGIS')
gdf

In [ ]:
gdf = gpd.GeoDataFrame(gdf, crs="EPSG:4326")
gdf["lat"]=gdf['geometry'].y
gdf ["lon"] = gdf['geometry'].x

gdf

In [ ]:
px.set_mapbox_access_token(open("mapboxtoken").read())
fig = px.line_mapbox(gdf,
                        lat=gdf.geometry.y,
                        lon=gdf.geometry.x,
                        hover_name="address",
                       
                        )

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=6, mapbox_center_lat = 29,
    margin={"r":0,"t":0,"l":0,"b":0})



fig.show()